In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import scipy.interpolate

In [2]:
# --- 0. Define Constants ---
# Gear ratio from tachometer to turbine
GEAR_RATIO = 5.095238095
# Nominal bearing fault orders (cycles per revolution)
# [cage, ball, inner race, outer race]
FAULT_ORDERS_BPF = [0.43, 7.05, 10.78, 8.22]
# Vibration signal sample rate
SAMPLE_RATE_HZ = 93750.0

In [3]:
filenames = ['file_01.csv',
'file_02.csv',
'file_03.csv',
'file_04.csv',
'file_05.csv',
'file_06.csv',
'file_07.csv',
'file_08.csv',
'file_09.csv',
'file_10.csv',
'file_11.csv',
'file_12.csv',
'file_13.csv',
'file_14.csv',
'file_15.csv',
'file_16.csv',
'file_17.csv',
'file_18.csv',
'file_19.csv',
'file_20.csv',
'file_21.csv',
'file_22.csv',
'file_23.csv',
'file_24.csv',
'file_25.csv',
'file_26.csv',
'file_27.csv',
'file_28.csv',
'file_29.csv',
'file_30.csv',
'file_31.csv',
'file_32.csv',
'file_33.csv',
'file_34.csv',
'file_35.csv',
'file_36.csv',
'file_37.csv',
'file_38.csv',
'file_39.csv',
'file_40.csv',
'file_41.csv',
'file_42.csv',
'file_43.csv',
'file_44.csv',
'file_45.csv',
'file_46.csv',
'file_47.csv',
'file_48.csv',
'file_49.csv',
'file_50.csv',
'file_51.csv',
'file_52.csv',
'file_53.csv']

In [4]:
# Dictionaries to store results
analysis_results = {}
plot_files = []
scoring_results = []

In [6]:
# --- 1. First Pass: Analyze Speed Stability ---
for filename in filenames:
    print(f"\nAnalyzing {filename}...")

    # Load data
    try:
        df = pd.read_csv(filename)
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        continue

    # Extract vibration signal
    v_signal = df['v'].values

    # Extract sparse tachometer timestamps and drop NaNs
    zct_times = df['zct'].dropna().values

    if len(zct_times) < 2:
        print(f"File {filename} has insufficient tachometer data. Skipping.")
        analysis_results[filename] = {'error': 'Insufficient tach data'}
        continue

    # Calculate time between tach pulses (periods)
    tach_periods = np.diff(zct_times)

    # Calculate instantaneous tachometer frequency (1 / period)
    # We assume 1 pulse per revolution (PPR) for the tachometer
    f_tach_inst = 1.0 / tach_periods

    # Calculate instantaneous turbine frequency
    f_turbine_inst = f_tach_inst * GEAR_RATIO

    # Create a time vector for these frequency values, placing them at the midpoint of each period
    t_freq_axis = zct_times[:-1] + tach_periods / 2.0

    # Calculate speed statistics
    mean_speed_hz = np.mean(f_turbine_inst)
    min_speed_hz = np.min(f_turbine_inst)
    max_speed_hz = np.max(f_turbine_inst)
    speed_variation_percent = 100.0 * (max_speed_hz - min_speed_hz) / mean_speed_hz

    # Store results
    analysis_results[filename] = {
        'mean_turbine_speed_hz': mean_speed_hz,
        'min_turbine_speed_hz': min_speed_hz,
        'max_turbine_speed_hz': max_speed_hz,
        'speed_variation_percent': speed_variation_percent,
        'num_v_samples': len(v_signal),
        'num_tach_pulses': len(zct_times),
        'data_duration_sec': len(v_signal) / SAMPLE_RATE_HZ
    }

    # --- Plot Instantaneous Speed ---
    plt.figure(figsize=(10, 6))
    plt.plot(t_freq_axis, f_turbine_inst)
    plt.title(f'Instantaneous Turbine Speed for {filename}\nMean: {mean_speed_hz:.2f} Hz, Variation: {speed_variation_percent:.2f}%')
    plt.xlabel('Time (s)')
    plt.ylabel('Turbine Speed (Hz)')
    plt.grid(True)
    plt.axhline(mean_speed_hz, color='r', linestyle='--', label=f'Mean ({mean_speed_hz:.2f} Hz)')
    plt.legend()
    plot_filename = f'speed_analysis_{filename}.png'
    plt.savefig(plot_filename)
    plt.close()
    plot_files.append(plot_filename)



Analyzing file_01.csv...

Analyzing file_02.csv...

Analyzing file_03.csv...

Analyzing file_04.csv...

Analyzing file_05.csv...

Analyzing file_06.csv...

Analyzing file_07.csv...

Analyzing file_08.csv...

Analyzing file_09.csv...

Analyzing file_10.csv...

Analyzing file_11.csv...

Analyzing file_12.csv...

Analyzing file_13.csv...

Analyzing file_14.csv...

Analyzing file_15.csv...

Analyzing file_16.csv...

Analyzing file_17.csv...

Analyzing file_18.csv...

Analyzing file_19.csv...

Analyzing file_20.csv...

Analyzing file_21.csv...

Analyzing file_22.csv...

Analyzing file_23.csv...

Analyzing file_24.csv...

Analyzing file_25.csv...

Analyzing file_26.csv...

Analyzing file_27.csv...

Analyzing file_28.csv...

Analyzing file_29.csv...

Analyzing file_30.csv...

Analyzing file_31.csv...

Analyzing file_32.csv...

Analyzing file_33.csv...

Analyzing file_34.csv...

Analyzing file_35.csv...

Analyzing file_36.csv...

Analyzing file_37.csv...

Analyzing file_38.csv...

Analyzing f

In [7]:
# Print summary of speed analysis
print("\n\n--- Speed Variation Analysis Summary ---")
for filename, results in analysis_results.items():
    if 'error' in results:
        print(f"{filename}: {results['error']}")
    else:
        print(f"{filename}:")
        print(f"  - Mean Turbine Speed: {results['mean_turbine_speed_hz']:.2f} Hz (Nominal: 536.27 Hz)")
        print(f"  - Speed Variation: {results['speed_variation_percent']:.2f} %")
        print(f"  - Data Duration: {results['data_duration_sec']:.2f} s")
        print(f"  - Tach Pulses: {results['num_tach_pulses']}")

print(f"\n--- Speed analysis plots generated: {plot_files} ---")



--- Speed Variation Analysis Summary ---
file_01.csv:
  - Mean Turbine Speed: 536.02 Hz (Nominal: 536.27 Hz)
  - Speed Variation: 0.32 %
  - Data Duration: 2.00 s
  - Tach Pulses: 529
file_02.csv:
  - Mean Turbine Speed: 536.62 Hz (Nominal: 536.27 Hz)
  - Speed Variation: 0.24 %
  - Data Duration: 2.00 s
  - Tach Pulses: 530
file_03.csv:
  - Mean Turbine Speed: 536.27 Hz (Nominal: 536.27 Hz)
  - Speed Variation: 0.50 %
  - Data Duration: 2.00 s
  - Tach Pulses: 529
file_04.csv:
  - Mean Turbine Speed: 536.21 Hz (Nominal: 536.27 Hz)
  - Speed Variation: 0.18 %
  - Data Duration: 2.00 s
  - Tach Pulses: 529
file_05.csv:
  - Mean Turbine Speed: 536.75 Hz (Nominal: 536.27 Hz)
  - Speed Variation: 0.25 %
  - Data Duration: 2.00 s
  - Tach Pulses: 530
file_06.csv:
  - Mean Turbine Speed: 535.90 Hz (Nominal: 536.27 Hz)
  - Speed Variation: 0.20 %
  - Data Duration: 2.00 s
  - Tach Pulses: 528
file_07.csv:
  - Mean Turbine Speed: 536.55 Hz (Nominal: 536.27 Hz)
  - Speed Variation: 0.29 %
  -

In [8]:
# --- 2. Second Pass: Angular Resampling and Envelope Order Analysis ---
print("\n\n--- Proceeding with Angular Resampling and Envelope Order Analysis ---")

for filename in filenames:
    print(f"\nProcessing {filename} for degradation scoring...")

    if 'error' in analysis_results[filename]:
        print(f"Skipping {filename} due to previous error.")
        continue

    # --- 2.1 Load Data ---
    df = pd.read_csv(filename)
    v_signal = df['v'].values
    zct_times = df['zct'].dropna().values

    # Create time vector for vibration signal
    t_v_axis = np.arange(len(v_signal)) / SAMPLE_RATE_HZ

    # --- 2.2 Create Phase-Time Map ---
    # We assume 1 pulse/rev, so each zct_time is a 2*pi radians (360 deg) marker
    # We will create an unwrapped phase vector
    phi_at_zct = np.arange(len(zct_times)) * 2 * np.pi  # Phase in radians

    # Create an interpolator to find the phase at any given time `t`
    # This linearly interpolates the phase between tach pulses
    phase_interpolator = scipy.interpolate.interp1d(
        zct_times,
        phi_at_zct,
        kind='linear',
        fill_value="extrapolate" # Extrapolate to cover edges of the signal
    )

    # Calculate the (unevenly spaced) instantaneous phase for *every* vibration sample
    phi_v_unwrapped = phase_interpolator(t_v_axis)

    # --- 2.3 Perform Angular Resampling ---
    # We now have v(t) and phi(t). We want v(phi).
    # We will create a *new*, *evenly spaced* angular axis
    # The number of samples will be the same, for simplicity
    phi_regular = np.linspace(
        phi_v_unwrapped.min(),
        phi_v_unwrapped.max(),
        len(phi_v_unwrapped)
    )

    # Interpolate the vibration signal `v_signal` from the
    # *uneven* `phi_v_unwrapped` grid onto the *even* `phi_regular` grid
    # This is the core of angular resampling
    v_resampled = np.interp(phi_regular, phi_v_unwrapped, v_signal)

    # --- 2.4 Envelope Analysis in Order Domain ---

    # We must bandpass filter in the angular (order) domain.
    # We need to find a high-frequency resonance band.
    # We'll use the nominal speed to translate the Hz band to an Order band.
    mean_speed_hz = analysis_results[filename]['mean_turbine_speed_hz']

    # Let's target a resonance band of [20 kHz, 40 kHz]
    band_low_hz = 20000.0
    band_high_hz = 40000.0

    # Convert Hz to Order (Order = Freq / Shaft_Speed)
    band_low_order = band_low_hz / mean_speed_hz
    band_high_order = band_high_hz / mean_speed_hz

    # We need the "sampling rate" in the angular domain.
    # This is "samples per revolution".
    total_revolutions = (phi_regular[-1] - phi_regular[0]) / (2 * np.pi)
    num_samples = len(phi_regular)
    samples_per_rev_avg = num_samples / total_revolutions

    # Nyquist order is samples_per_rev_avg / 2
    nyquist_order = samples_per_rev_avg / 2.0

    # Check if our band is valid
    if band_high_order >= nyquist_order:
        print(f"Warning: Desired high band {band_high_order:.1f} exceeds Nyquist order {nyquist_order:.1f}. Clamping.")
        band_high_order = nyquist_order * 0.99

    # Create a 4th-order Butterworth bandpass filter
    try:
        b, a = scipy.signal.butter(
            4,
            [band_low_order, band_high_order],
            btype='band',
            fs=samples_per_rev_avg
        )
    except ValueError as e:
        print(f"Error creating filter for {filename}: {e}. Skipping file.")
        print(f"Filter params: low={band_low_order}, high={band_high_order}, fs={samples_per_rev_avg}")
        scoring_results.append({'file': filename, 'score': 0, 'error': str(e)})
        continue

    # Apply the filter to the angle-resampled signal
    v_resampled_filtered = scipy.signal.lfilter(b, a, v_resampled)

    # --- 2.5 Calculate Envelope & Order Spectrum ---

    # Calculate the analytic signal (Hilbert transform) on the *filtered* signal
    analytic_signal = scipy.signal.hilbert(v_resampled_filtered)

    # Get the envelope (magnitude of the analytic signal)
    envelope = np.abs(analytic_signal)

    # Remove DC component (mean)
    envelope_demeaned = envelope - np.mean(envelope)

    # Calculate the Power Spectrum of the envelope
    # This is the "Envelope Order Spectrum"
    env_spectrum_power = np.abs(np.fft.fft(envelope_demeaned))**2

    # Create the Order axis for the spectrum
    # The 'frequency' in fftfreq is cycles/sample.
    # We want cycles/revolution (Order).
    # Order = (cycles / sample) * (samples / revolution)
    order_axis = np.fft.fftfreq(num_samples) * samples_per_rev_avg

    # Keep only the positive half of the spectrum
    positive_mask = order_axis >= 0
    orders = order_axis[positive_mask]
    spectrum_power = env_spectrum_power[positive_mask]
    spectrum_amplitude = np.sqrt(spectrum_power) # Convert power to amplitude

    # --- 2.6 Score the Spectrum ---
    # We will score based on the peak amplitude at the known fault orders
    score = 0.0
    fault_peak_details = {}

    # We look for the peak in a small band around each fault order
    search_width_order = 0.1 # Look in a band of +/- 0.05 orders

    for fault_name, fault_order in zip(['Cage', 'Ball', 'InnerRace', 'OuterRace'], FAULT_ORDERS_BPF):
        order_min = fault_order - search_width_order / 2.0
        order_max = fault_order + search_width_order / 2.0

        # Find indices within this band
        band_indices = np.where((orders >= order_min) & (orders <= order_max))[0]

        if len(band_indices) > 0:
            # Find the max amplitude in this band
            peak_amplitude_in_band = np.max(spectrum_amplitude[band_indices])

            # Add this peak amplitude to the total score
            # We weight the inner/outer race faults higher as they are more common indicators
            weight = 1.0
            if fault_name == 'InnerRace':
                weight = 2.0
            elif fault_name == 'OuterRace':
                weight = 2.0

            score += peak_amplitude_in_band * weight
            fault_peak_details[fault_name] = peak_amplitude_in_band
        else:
            fault_peak_details[fault_name] = 0.0

    scoring_results.append({
        'file': filename,
        'score': score,
        'peaks': fault_peak_details
    })

    # --- 2.7 Plot the Envelope Order Spectrum ---
    plt.figure(figsize=(12, 7))
    plt.plot(orders, spectrum_amplitude) # Plot amplitude

    # Add vertical lines for fault orders
    colors = ['g', 'b', 'r', 'purple']
    for (fault_name, fault_order), color in zip(zip(['Cage', 'Ball', 'InnerRace', 'OuterRace'], FAULT_ORDERS_BPF), colors):
        plt.axvline(fault_order, color=color, linestyle='--', label=f'{fault_name} ({fault_order})')

    plt.title(f'Envelope Order Spectrum for {filename}\nScore: {score:.2f}')
    plt.xlabel('Order (cycles / turbine revolution)')
    plt.ylabel('Envelope Amplitude')
    plt.legend()
    plt.grid(True)
    plt.xlim(0, 20) # Zoom in on the area of interest

    # Find a reasonable Y-limit
    try:
        # Calculate 99.5th percentile of amplitude in the 0-20 order range
        y_lim_max = np.percentile(spectrum_amplitude[orders <= 20], 99.5)
        plt.ylim(0, y_lim_max * 1.1) # Set Y-limit to 110% of 99.5th percentile
    except:
        pass # Failsafe if percentile calc fails

    spectrum_plot_file = f'envelope_order_spectrum_{filename}.png'
    plt.savefig(spectrum_plot_file)
    plt.close()
    plot_files.append(spectrum_plot_file)



--- Proceeding with Angular Resampling and Envelope Order Analysis ---

Processing file_01.csv for degradation scoring...

Processing file_02.csv for degradation scoring...

Processing file_03.csv for degradation scoring...

Processing file_04.csv for degradation scoring...

Processing file_05.csv for degradation scoring...

Processing file_06.csv for degradation scoring...

Processing file_07.csv for degradation scoring...

Processing file_08.csv for degradation scoring...

Processing file_09.csv for degradation scoring...

Processing file_10.csv for degradation scoring...

Processing file_11.csv for degradation scoring...

Processing file_12.csv for degradation scoring...

Processing file_13.csv for degradation scoring...

Processing file_14.csv for degradation scoring...

Processing file_15.csv for degradation scoring...

Processing file_16.csv for degradation scoring...

Processing file_17.csv for degradation scoring...

Processing file_18.csv for degradation scoring...

Processi

In [9]:
# --- 3. Final Ranking ---
print("\n\n--- Final Degradation Ranking ---")

# Sort by score, ascending (lower score = less degradation)
try:
    ranked_files = sorted(scoring_results, key=lambda x: x.get('score', 0))

    print("Ranked files from LEAST degraded to MOST degraded (Predicted Timeline):")
    for i, result in enumerate(ranked_files):
        print(f"  Rank {i+1}: {result['file']} (Score: {result.get('score', 'N/A'):.2f})")
        if 'peaks' in result:
            print(f"      - InnerRace Peak: {result['peaks'].get('InnerRace', 0):.2f}")
            print(f"      - OuterRace Peak: {result['peaks'].get('OuterRace', 0):.2f}")
            print(f"      - Ball Peak: {result['peaks'].get('Ball', 0):.2f}")
            print(f"      - Cage Peak: {result['peaks'].get('Cage', 0):.2f}")

except Exception as e:
    print(f"Error during final ranking: {e}")
    print("Raw scoring results:")
    for result in scoring_results:
        print(result)

print(f"\nAll generated plots: {plot_files}")
print("Analysis complete.")



--- Final Degradation Ranking ---
Ranked files from LEAST degraded to MOST degraded (Predicted Timeline):
  Rank 1: file_06.csv (Score: 119235.33)
      - InnerRace Peak: 16926.75
      - OuterRace Peak: 18260.25
      - Ball Peak: 21262.67
      - Cage Peak: 27598.67
  Rank 2: file_36.csv (Score: 122884.92)
      - InnerRace Peak: 16215.58
      - OuterRace Peak: 21479.32
      - Ball Peak: 17997.10
      - Cage Peak: 29498.03
  Rank 3: file_16.csv (Score: 123218.38)
      - InnerRace Peak: 15251.66
      - OuterRace Peak: 24463.48
      - Ball Peak: 19582.70
      - Cage Peak: 24205.38
  Rank 4: file_15.csv (Score: 124107.84)
      - InnerRace Peak: 14438.24
      - OuterRace Peak: 22701.58
      - Ball Peak: 28759.96
      - Cage Peak: 21068.23
  Rank 5: file_26.csv (Score: 126392.04)
      - InnerRace Peak: 13188.90
      - OuterRace Peak: 22466.43
      - Ball Peak: 27512.87
      - Cage Peak: 27568.51
  Rank 6: file_46.csv (Score: 126551.59)
      - InnerRace Peak: 15119.44
   